In [ ]:
!wget https://students.mimuw.edu.pl/~pk394395/data2.tgz
!mkdir data
!tar xvf data2.tgz -C data/
!ls data

--2020-12-04 15:05:11--  https://students.mimuw.edu.pl/~pk394395/data2.tgz
Resolving students.mimuw.edu.pl (students.mimuw.edu.pl)... 193.0.96.129, 2001:6a0:5001:1::3
Connecting to students.mimuw.edu.pl (students.mimuw.edu.pl)|193.0.96.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1556480 (1.5M) [application/x-gzip]
Saving to: ‘data2.tgz’

data2.tgz           100%[===================>]   1.48M  2.42MB/s    in 0.6s    

2020-12-04 15:05:12 (2.42 MB/s) - ‘data2.tgz’ saved [1556480/1556480]

metal2/metal0.jpg
metal2/metal100.jpg
metal2/metal101.jpg
metal2/metal102.jpg
metal2/metal103.jpg
metal2/metal104.jpg
metal2/metal105.jpg
metal2/metal106.jpg
metal2/metal107.jpg
metal2/metal108.jpg
metal2/metal109.jpg
metal2/metal10.jpg
metal2/metal110.jpg
metal2/metal111.jpg
metal2/metal112.jpg
metal2/metal113.jpg
metal2/metal114.jpg
metal2/metal115.jpg
metal2/metal116.jpg
metal2/metal117.jpg
metal2/metal118.jpg
metal2/metal119.jpg
metal2/metal11.jpg
metal2/metal120.jp

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_width, img_height = (224, 224)
batch_size = 8

train_datagen = ImageDataGenerator(
  # rescale=1./255,
  rotation_range=40, 
  width_shift_range=0.2, 
  height_shift_range=0.2, 
  shear_range=0.2, 
  zoom_range=0.2, 
  horizontal_flip=True, 
  fill_mode='nearest',
  validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
  "data/",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  subset='training'
)

validation_generator = train_datagen.flow_from_directory(
  "data/",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  subset='validation'
)


all_datagen = ImageDataGenerator(rescale=1./255,
  rotation_range=40, 
  width_shift_range=0.2, 
  height_shift_range=0.2, 
  shear_range=0.2, 
  zoom_range=0.2, 
  horizontal_flip=True, 
  fill_mode='nearest',
)

all_generator = all_datagen.flow_from_directory(
    "data/",
    target_size=(img_height, img_width),
    batch_size=batch_size
)

Found 311 images belonging to 3 classes.
Found 77 images belonging to 3 classes.
Found 388 images belonging to 3 classes.


In [ ]:
import tensorflow as tf
import tensorflow.keras.applications

mobile_net = tf.keras.applications.MobileNetV2(classes=3, include_top=False, input_shape=(224, 224, 3))

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras import optimizers

base_model = mobile_net

x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
y = Dense(3, activation='softmax', kernel_regularizer='l2')(x)
model = Model(inputs=base_model.input, outputs=y)

for layer in base_model.layers:
  layer.trainable = Falserms

model.compile(optimizer=optimizers.RMSProp(lr=10e-5), loss='categorical_crossentropy', metrics=['acc'])
#model.summary()

In [ ]:
#class_weight = {
#    0: 224 / 101,
#    1: 224 / 56,
#    2: 224 / 67,
#}
history = model.fit_generator(train_generator, epochs=30, validation_data=validation_generator)#, validation_steps=5, steps_per_epoch=16)#, class_weight=class_weight)
#history = model.fit_generator(all_generator, epochs=50)#, class_weight=class_weight)

Epoch 1/30
39/39 [==============================] - 5s 128ms/step - loss: 0.9765 - acc: 0.7492 - val_loss: 1.1666 - val_acc: 0.7662
Epoch 2/30
39/39 [==============================] - 4s 109ms/step - loss: 0.4976 - acc: 0.8489 - val_loss: 0.5781 - val_acc: 0.8831
Epoch 3/30
39/39 [==============================] - 4s 108ms/step - loss: 0.4489 - acc: 0.8650 - val_loss: 0.2959 - val_acc: 0.8961
Epoch 4/30
39/39 [==============================] - 4s 109ms/step - loss: 0.3572 - acc: 0.8842 - val_loss: 0.4702 - val_acc: 0.8961
Epoch 5/30
39/39 [==============================] - 4s 109ms/step - loss: 0.3695 - acc: 0.8842 - val_loss: 0.2761 - val_acc: 0.9221
Epoch 6/30
39/39 [==============================] - 4s 109ms/step - loss: 0.3589 - acc: 0.8810 - val_loss: 0.3228 - val_acc: 0.8831
Epoch 7/30
39/39 [==============================] - 4s 109ms/step - loss: 0.3477 - acc: 0.8875 - val_loss: 0.2625 - val_acc: 0.9610
Epoch 8/30
39/39 [==============================] - 4s 109ms/step - loss: 0.

In [ ]:
# for layer in model.layers[:-23]:
#    layer.trainable = False

# # Last two blocks
# for layer in model.layers[-23:]:
#    layer.trainable = True

# model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])#, 'bacc'])

# class_weight = {
#     0: 224 / 101,
#     1: 224 / 56,
#     2: 224 / 67,
# }
# history = model.fit_generator(train_generator, epochs=50, validation_data=validation_generator, class_weight=class_weight)

In [ ]:
import tensorflow.saved_model

tensorflow.saved_model.save(model, "garbage_mobilenet_v2_dense_fine_tune")

INFO:tensorflow:Assets written to: garbage_mobilenet_v2_dense_fine_tune/assets


INFO:tensorflow:Assets written to: garbage_mobilenet_v2_dense_fine_tune/assets


In [ ]:
from tensorflow.lite import *
converter = tensorflow.lite.TFLiteConverter.from_saved_model("garbage_mobilenet_v2_dense_fine_tune")
tflite_model = converter.convert()

# Save the model.
with open('garbage_mobilenet_v2_dense_fine_tune.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from google.colab import files
files.download('garbage_mobilenet_v2_dense_fine_tune.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>